## 데이터

In [1]:
# 셀레니움 잘 깔렸는지 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.12.10
4.39.0


In [2]:
import os
import requests
import pandas as pd
import sqlite3
import re
import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller  # setup chrome options

import urllib.request
from urllib.request import urlretrieve
import time

import warnings
warnings.filterwarnings('ignore')

# 프로세스

In [3]:
TITLE = "Remote Sensing of Environment" # 원하는 저널명
TITLE = TITLE.replace(" ", "+")  # 공백을 +로 변경
START_PAGE = 1               # 재시작할 페이지 번호 (예: 1)
DOWNLOAD_WAIT_SECONDS = 270   # 대량 PDF 다운로드 대기 시간 (약 4분 30초)
PAGE_CHANGE_DELAY = 5        # 페이지 이동 후 안정화 대기 시간 (약 5초)
MAX_PAGE_VISITS = 300         # 안전장치: 최대 3페이지 처리
SET_YEAR = "2001"

# 기관 로그인

In [4]:
from selenium.webdriver.chrome.webdriver import WebDriver

import pandas as pd
import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
import re

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")

# 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
wait = WebDriverWait(driver, 5)  # 최대 5초까지 기다림

# User-Agent 설정으로 봇 감지 회피
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# 대학 기관 로그인 - credentials.json에서 읽기
with open('credentials.json', 'r', encoding='utf-8') as f:
    credentials = json.load(f)
    
univ_url = "https://lib.gachon.ac.kr/login"
univ_id = credentials['univ_id']
univ_pw = credentials['univ_pw']

print("=" * 50)
print("기관 로그인 진행 중...")
print("=" * 50)

# 로그인 페이지 접속
print(f"로그인 페이지 접속 중: {univ_url}")
driver.get(univ_url)
wait.until(EC.presence_of_element_located((By.ID, "id")))

try:
    # ID 입력
    print("ID 입력 필드 찾는 중...")
    id_input = wait.until(EC.presence_of_element_located((By.ID, "id")))
    id_input.clear()
    id_input.send_keys(univ_id)
    print(f"  ✓ ID 입력 완료: {univ_id}")
    
    # 비밀번호 입력
    print("비밀번호 입력 필드 찾는 중...")
    pw_input = wait.until(EC.presence_of_element_located((By.ID, "password")))
    pw_input.clear()
    pw_input.send_keys(univ_pw)
    print("  ✓ 비밀번호 입력 완료")
    
    # 로그인 버튼 찾기 및 클릭
    print("로그인 버튼 찾는 중...")
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']")))
    driver.execute_script("arguments[0].click();", login_button)
    print("  ✓ 로그인 버튼 클릭 완료")
    
    # 로그인 완료 대기
    wait.until(lambda d: "login" not in d.current_url.lower() or d.current_url != univ_url)
    print("\n✅ 로그인 완료!")
    print(f"현재 URL: {driver.current_url}")
    
except Exception as e:
    print(f"❌ 로그인 중 오류 발생: {e}")
    print("로그인을 수동으로 진행해주세요.")


# Advanced Search 페이지 접속
print("\n" + "=" * 50)
print("Advanced Search 페이지 접속 중...")
print("=" * 50)
url = f'https://www-sciencedirect-com-ssl.glibproxy.gachon.ac.kr/search?pub={TITLE}&date={SET_YEAR}&articleTypes=FLA&lastSelectedFacet=articleTypes'
print(f"페이지 접속 중: {url}")
driver.get(url)
# 페이지 로드 대기 (입력 필드가 나타날 때까지)
try:
    wait.until(EC.presence_of_element_located((By.ID, "pub")))
    print("  ✓ Advanced Search 페이지 로드 완료")
except:
    print("  ⚠ 페이지 로드 대기 중 타임아웃 발생")

print("\n페이지 준비 완료. 이제 크롤링을 시작합니다.")

기관 로그인 진행 중...
로그인 페이지 접속 중: https://lib.gachon.ac.kr/login
ID 입력 필드 찾는 중...
  ✓ ID 입력 완료: westhi320
비밀번호 입력 필드 찾는 중...
  ✓ 비밀번호 입력 완료
로그인 버튼 찾는 중...
  ✓ 로그인 버튼 클릭 완료

✅ 로그인 완료!
현재 URL: https://lib.gachon.ac.kr/

Advanced Search 페이지 접속 중...
페이지 접속 중: https://www-sciencedirect-com-ssl.glibproxy.gachon.ac.kr/search?pub=Remote+Sensing+of+Environment&date=2001&articleTypes=FLA&lastSelectedFacet=articleTypes
  ✓ Advanced Search 페이지 로드 완료

페이지 준비 완료. 이제 크롤링을 시작합니다.


# Science Direct 논문 크롤링

In [ ]:
print("\n" + "=" * 50)
print("검색 결과 페이지에서 크롤링 시작")
print("=" * 50)

def get_current_page():
    """현재 페이지 번호를 가져온다."""
    try:
        # "Page 1 of 5" 형식의 텍스트 찾기
        page_text = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Page') and contains(text(), 'of')]"))
        )
        text = page_text.text
        # "Page 1 of 5"에서 첫 번째 숫자 추출
        match = re.search(r'Page\s+(\d+)\s+of', text)
        if match:
            return int(match.group(1))
    except:
        pass
    # 기본값으로 1 반환
    return 1

def navigate_to_target_page(target_page: int):
    """Next 버튼을 계속 누르면서 원하는 페이지까지 이동한다."""
    if target_page < 1:
        raise ValueError("target_page는 1 이상의 정수여야 합니다.")
    
    current_page = get_current_page()
    
    if target_page == current_page:
        print(f"  ✓ 이미 {target_page} 페이지에 있습니다.")
        return target_page
    
    if target_page < current_page:
        print(f"  ⚠ 목표 페이지({target_page})가 현재 페이지({current_page})보다 작습니다. 현재 페이지에서 시작합니다.")
        return current_page
    
    print(f"\n▶ START_PAGE={target_page}로 이동 준비...")
    print(f"  현재 페이지: {current_page}, 목표 페이지: {target_page}")
    
    # Next 버튼을 계속 클릭하여 목표 페이지까지 이동
    while current_page < target_page:
        try:
            next_button = WebDriverWait(driver, 8).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-aa-name='srp-next-page']"))
            )
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", next_button)
            driver.execute_script("arguments[0].click();", next_button)
            current_page += 1
            print(f"  → Next 버튼 클릭 ({current_page} 페이지로 이동)")
            time.sleep(PAGE_CHANGE_DELAY)
            
            WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")
            time.sleep(1)
            
            # 현재 페이지 확인
            current_page = get_current_page()
        except TimeoutException:
            print("  ⚠ Next 버튼을 찾지 못했습니다. 마지막 페이지로 판단합니다.")
            break
    
    print(f"▶ {current_page} 페이지에서 다운로드를 재개합니다.")
    return current_page


def select_all_results():
    """현재 페이지에서 결과 전체 선택 체크박스를 클릭한다."""
    checkbox = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "select-all-results"))
    )
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", checkbox)
    time.sleep(0.5)
    
    # aria-checked 속성 확인 또는 is_selected() 확인
    is_checked = checkbox.get_attribute("aria-checked") == "true" or checkbox.is_selected()
    if not is_checked:
        driver.execute_script("arguments[0].click();", checkbox)
        time.sleep(0.5)
        # 클릭 후 다시 확인
        is_checked = checkbox.get_attribute("aria-checked") == "true" or checkbox.is_selected()
        if not is_checked:
            # 일반 클릭 시도
            checkbox.click()
            time.sleep(0.5)
    print("  ✓ 전체 결과 선택 완료")
    time.sleep(1)


def trigger_download():
    """현재 페이지에서 선택된 논문을 일괄 다운로드한다."""
    # span.download-all-link-text를 찾아서 부모 요소(button) 클릭
    download_span = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "span.download-all-link-text"))
    )
    # 부모 button 요소 찾기
    download_button = download_span.find_element(By.XPATH, "./ancestor::button")
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", download_button)
    driver.execute_script("arguments[0].click();", download_button)
    print("  ✓ 다운로드 버튼 클릭")
    time.sleep(DOWNLOAD_WAIT_SECONDS)


def process_current_page(page_number: int):
    print(f"\n--- {page_number} 페이지 처리 시작 ---")
    # 1) 현재 페이지 전체 선택
    select_all_results()
    # 2) 선택된 논문 일괄 다운로드
    trigger_download()
    print(f"--- {page_number} 페이지 처리 완료 ---")


def go_to_next_page(current_page: int):
    """다음 페이지로 이동한다."""
    try:
        next_button = WebDriverWait(driver, 8).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-aa-name='srp-next-page']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", next_button)
        driver.execute_script("arguments[0].click();", next_button)
        print(f"  → Next 버튼 클릭")
        time.sleep(PAGE_CHANGE_DELAY)
        
        WebDriverWait(driver, 20).until(lambda d: d.execute_script("return document.readyState") == "complete")
        time.sleep(1)
        
        # 현재 페이지 확인
        next_page = get_current_page()
        return next_page
    except TimeoutException:
        print("  ⚠ Next 버튼을 찾지 못했습니다. 마지막 페이지로 판단합니다.")
        return None


current_page = get_current_page()
visited_pages = 0

if START_PAGE < 1:
    raise ValueError("START_PAGE는 1 이상의 정수여야 합니다.")

if START_PAGE != current_page:
    current_page = navigate_to_target_page(START_PAGE)

while visited_pages < MAX_PAGE_VISITS:
    process_current_page(current_page)
    visited_pages += 1

    next_page_number = go_to_next_page(current_page)
    if next_page_number is None:
        print("\n✅ 더 이상 이동할 페이지가 없어 작업을 종료합니다.")
        break

    current_page = next_page_number
else:
    print(f"\n⚠ 안전장치(MAX_PAGE_VISITS={MAX_PAGE_VISITS})에 도달하여 루프를 종료합니다.")


검색 결과 페이지에서 크롤링 시작

--- 1 페이지 처리 시작 ---
  ✓ 전체 결과 선택 완료
  ✓ 다운로드 버튼 클릭
